# Tavily Search

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

## Overview

### Integration details
| Class | Package | Serializable | [JS support](https://js.langchain.com/docs/integrations/tools/tavily_search) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [TavilySearchResults](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ❌ | ✅ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ✅ | ✅ | Title, URL, raw content, answer, images, image descriptions | 1,000 free searches / month | 


## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package.

In [ ]:
%pip install -qU "langchain-community>=0.3.18" tavily-python

### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [1]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
# if you experience issues with api keys try to set the api key manually
# os.environ["TAVILY_API_KEY"] = "<YOUR_API_KEY>"

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [11]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instantiate an instance of the Tavily search tools. The tool accepts various parameters to customize the search:

In [28]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    topic="general",
    include_answer=False,
    include_raw_content=True,           # including raw content may lead to hitting context length limits
    # include_images=True,
    # include_image_descriptions=True
    # time_range="day",
    # days=3,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

The `TavilySearchResults` tool accepts the following arguments:
- `query` (required): A natural language search query
- `search_depth` (optional): The depth of the search ("basic" or "advanced")
- `time_range` (optional): Time range filter ("day", "week", "month", "year")
- `include_domains` (optional): List of domains to limit search results to
- `exclude_domains` (optional): List of domains to exclude from search results
- `include_images` (optional): Whether to include query-related images in response

NOTE: The optional arguments are available for ReAct agents to dynamically set, if you set a argument during instantiation and then Invoke the tool with a different value, the tool will use the value you passed during invokation. To read more about ReAct agents, check out the [ReAct agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/) documentation.

In [10]:
# Basic query
tool.invoke({"query": "What happened at the last wimbledon"})
# Query with time range and domain filtering
# tool.invoke({
#     "query": "latest tennis news",
#     "time_range": "day",
#     "include_domains": ["espn.com", "tennis.com"]
# })

{'query': 'What happened at the last wimbledon',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': "Andy Murray pulls out of the men's singles draw at his last Wimbledon",
   'url': 'https://www.nbcnews.com/news/sports/andy-murray-wimbledon-tennis-singles-draw-rcna159912',
   'content': "NBC News Now LONDON — Andy Murray, one of the last decade's most successful male tennis players, has pulled out of the singles tournament at what is almost certain to be his last Wimbledon, his team confirmed Tuesday. Murray, 37, who has won the Wimbledon singles title twice and the U.S Open once, has been battling to be fit to play at the All England Club for weeks. “Unfortunately, despite working incredibly hard on his recovery since his operation just over a week ago, Andy has taken the very difficult decision not to play the singles this year,” his team said in a statement reported by Sky News. The news caps a glittering career on the ATP singles tour, which place

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [15]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2028 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"query": "euro 2028 host nation", "follow_up_questions": null, "answer": "The host nation for Euro 2028 is the UK and the Republic of Ireland. This decision was confirmed by the UEFA Executive Committee, with the event set to take place from 9 June to 9 July 2028.", "images": [], "results": [{"title": "UEFA Euro 2028 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2028", "content": 


## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [25]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [29]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("who won the super bowl this year")

AIMessage(content="The Philadelphia Eagles won the Super Bowl in 2025, defeating the Kansas City Chiefs with a final score of 40 to 22. Jalen Hurts, the Eagles' quarterback, was named the MVP of the game. This victory marked the Eagles' second Super Bowl win.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 15310, 'total_tokens': 15369, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a5e6599-9346-4c5d-af5d-34f3d810f46f-0', usage_metadata={'input_tokens': 15310, 'output_tokens': 59, 'total_tokens': 15369, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Here's the [LangSmith trace](https://smith.langchain.com/public/b43232c1-b243-4a7f-afeb-5fba8c84ba56/r) for this run.

## API reference

For detailed documentation of all TavilySearchResults features and configurations head to the API reference: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html